# LEET1307: Verbal Arithmetic Puzzle


Description omitted for copyright. Full problem statement found here:

<href>https://leetcode.com/problems/verbal-arithmetic-puzzle/

My solution (*solution1*) versus a few publically posted solutions.

In [1]:
import time

def solution1(words, result):
    # my solution    
    # search function
    def search(pos,col_sum,char_map):
        
        r,c = pos
        # reached end state, test if sum is zero
        if c==num_cols-1 and r==num_rows:
            return col_sum==0
        
        # reached final row, check if feasible solution 
        # if yes, move on
        # if no, return false
        if r==num_rows:
            if col_sum%10==0:
                return True and search((0,c+1),col_sum//10,char_map.copy())
            else:
                return(False)
            
        # check if letter in char_map
        # if yes, add to col_sum and continue
        if expr[r][c] in char_map.keys():
            return(search((r+1,c),col_sum + char_map[expr[r][c]]*[1,-1][r==num_rows-1],char_map.copy())==True)

        # else grab next avail number and continue
        else:
            avail_nums = set(range(10))- set([char_map[k] for k in char_map.keys() if k!='_'])
            
            while avail_nums:
                char_map[expr[r][c]] = avail_nums.pop()
                # if next iteration is true, return true
                if search((r+1,c),col_sum + char_map[expr[r][c]]*[1,-1][r==num_rows-1],char_map.copy())==True:
                    return(True)
                else:
                    # else, delete char_map entry and get next avail number
                    del char_map[expr[r][c]]
                
        return(False)
    
    # setup for search
    # get num or rows
    num_rows = len(words) + 1
    # get num of columns (max)
    num_cols = max([len(word) for word in words + [result]])
    
    #pad expression and reverse the order
    expr = [word[-1::-1]+'_'*(num_cols-len(word)+1) for word in words+[result]]
    # update num_cols
    num_cols = max([len(word) for word in words + [result]])

    pos = (0,0)
    col_sum = 0
    
    
    return(search(pos,col_sum,{'_':0}))



def solution2(words, result):
    # found here: https://leetcode.com/problems/verbal-arithmetic-puzzle/solutions/463921/python-backtracking-with-pruning-tricks/?orderBy=most_votes
    start = set()
    for word in words + [result]:
        if len(word) > 1:
            start.add(word[0])


    n = max(map(len, words + [result]))
    if len(result) < n:
        return False

    def dfs(idx, i, carry, visited, mp):
        if idx == n:
            return carry == 0
        if i == len(words) + 1:
            sums = sum(mp[word[-idx - 1]] if idx < len(word) else 0 for word in words) + carry
            if sums % 10 == mp[result[-idx - 1]]:
                carry = sums // 10
                return dfs(idx + 1, 0, carry, visited, mp)
            return False

        if (i < len(words) and idx >= len(words[i])):
            return dfs(idx, i + 1, carry, visited, mp)
        tmp = words + [result]
        ch = tmp[i][-idx-1]
        if ch in mp:
            return dfs(idx, i + 1, carry, visited, mp)
        begin = 0
        if ch in start:
            begin = 1
        for x in range(begin, 10):
            if x not in visited:
                visited.add(x)
                mp[ch] = x
                if dfs(idx, i + 1, carry, visited, mp.copy()):
                    return True
                visited.remove(x)
        return False

    return dfs(0, 0, 0, set(), {})



def solution3(words, result):
    # found here: https://leetcode.com/problems/verbal-arithmetic-puzzle/solutions/1216642/python3-backtracking/
    if max(map(len, words)) > len(result): return False # edge case 
    
    words.append(result)
    digits = [0]*10 
    mp = {} # mapping from letter to digit 
    
    def fn(i, j, val): 
        """Find proper mapping for words[i][~j] and result[~j] via backtracking."""
        if j == len(result): return val == 0 # base condition 
        if i == len(words): return val % 10 == 0 and fn(0, j+1, val//10)
        
        if j >= len(words[i]): return fn(i+1, j, val)
        if words[i][~j] in mp: 
            if j and j+1 == len(words[i]) and mp[words[i][~j]] == 0: return # backtrack (no leading 0)
            if i+1 == len(words): return fn(i+1, j, val - mp[words[i][~j]])
            else: return fn(i+1, j, val + mp[words[i][~j]])
        else: 
            for k, x in enumerate(digits): 
                if not x and (k or j == 0 or j+1 < len(words[i])): 
                    mp[words[i][~j]] = k
                    digits[k] = 1
                    if i+1 == len(words) and fn(i+1, j, val-k): return True 
                    if i+1 < len(words) and fn(i+1, j, val+k): return True 
                    digits[k] = 0
                    mp.pop(words[i][~j])
    
    return fn(0, 0, 0)

## Test Case 1

Result should be True.

In [2]:
words = ["SIX","SEVEN","SEVEN"]
result = "TWENTY"

print(f'solution1 = {solution1(words, result)}')
print(f'solution2 = {solution2(words, result)}')
print(f'solution3 = {solution3(words, result)}')

n_iter = 10**1

t0 = time.time()
for _ in range(n_iter):
    solution1(words,result)
t1 = time.time()
print(f'solution1 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution2(words,result)
t1 = time.time()
print(f'solution2 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution3(words,result)
t1 = time.time()
print(f'solution3 delta_t={t1-t0}')

solution1 = True
solution2 = True
solution3 = True
solution1 delta_t=1.890406847000122
solution2 delta_t=1.8332581520080566
solution3 delta_t=2.557828426361084


## Test Case 2

Result should be True.

In [3]:
words = ["SEND","MORE"]
result = "MONEY"

print(f'solution1 = {solution1(words, result)}')
print(f'solution2 = {solution2(words, result)}')
print(f'solution3 = {solution3(words, result)}')

n_iter = 10**1

t0 = time.time()
for _ in range(n_iter):
    solution1(words,result)
t1 = time.time()
print(f'solution1 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution2(words,result)
t1 = time.time()
print(f'solution2 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution3(words,result)
t1 = time.time()
print(f'solution3 delta_t={t1-t0}')

solution1 = True
solution2 = True
solution3 = True
solution1 delta_t=0.6189908981323242
solution2 delta_t=0.29612231254577637
solution3 delta_t=0.3291330337524414


## Test Case 3

Result should be False.

In [4]:
words = ["LEET","CODE"]
result = "POINT"

print(f'solution1 = {solution1(words, result)}')
print(f'solution2 = {solution2(words, result)}')
print(f'solution3 = {solution3(words.copy(), result)}')

n_iter = 10**1

t0 = time.time()
for _ in range(n_iter):
    solution1(words,result)
t1 = time.time()
print(f'solution1 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution2(words,result)
t1 = time.time()
print(f'solution2 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution3(words.copy(),result)
t1 = time.time()
print(f'solution3 delta_t={t1-t0}')

solution1 = False
solution2 = False
solution3 = None
solution1 delta_t=0.011763572692871094
solution2 delta_t=0.012234926223754883
solution3 delta_t=0.007528066635131836


## Test Case 4

Result should be True:
- 'a'=1, 'b'=0, so 
- 'aa'+'bb' = 'aa' => 11+00=11

In [5]:
words = ["AA","BB"]
result = "AA"

print(f'solution1 = {solution1(words, result)}')
print(f'solution2 = {solution2(words, result)}')
print(f'solution3 = {solution3(words, result)}')

n_iter = 10**1

t0 = time.time()
for _ in range(n_iter):
    solution1(words,result)
t1 = time.time()
print(f'solution1 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution2(words,result)
t1 = time.time()
print(f'solution2 delta_t={t1-t0}')

t0 = time.time()
for _ in range(n_iter):
    solution3(words.copy(),result)
t1 = time.time()
print(f'solution3 delta_t={t1-t0}')

solution1 = True
solution2 = False
solution3 = None
solution1 delta_t=0.004968404769897461
solution2 delta_t=0.0032279491424560547
solution3 delta_t=0.0043790340423583984
